# FAQ Notebook for Public Library Data

## What percentage of the US population is served by each of the budget categories of public libraries?

First, let's define our categories. The lowest budget categories are consolidated into one.

In [9]:
categories = [
  { "minValue": 30000000, "label": '$30M+' },
  { "minValue": 10000000, "maxValue": 30000000, "label": '$10M - $30M' },
  { "minValue": 5000000, "maxValue": 10000000, "label": '$5M - $10M' },
  { "minValue": 1000000, "maxValue": 5000000, "label": '$1M - $5M' },
  { "minValue": 400000, "maxValue": 1000000, "label": '$400K - $1M' },
  { "minValue": 200000, "maxValue": 400000, "label": '$200K - $400K' },
  { "minValue": 0, "maxValue": 200000, "label": 'Below $200K' }
]

Next we will read the public libary data.

In [10]:
import pandas as pd

lib_df = pd.read_csv("data/PLS_FY22_AE_pud22i.csv", encoding="latin-1")
lib_count = lib_df.shape[0]
print(f"Found {lib_count:,} entries from the public library survey")

Found 9,248 entries from the public library survey


Next we will categorize each library based on their budget value, then display the frequency/percent for each category

In [18]:
def get_category(lib, categories):
  category = "None"
  for cat in categories:
    if "minValue" in cat and "maxValue" in cat and lib["TOTINCM"] >= cat["minValue"] and lib["TOTINCM"] < cat["maxValue"]:
      category = cat["label"]
      break
    elif "minValue" in cat and lib["TOTINCM"] >= cat["minValue"]:
      category = cat["label"]
      break
  return category
lib_df["BUDGET_CATEGORY"] = lib_df.apply(lambda row: get_category(row, categories), axis=1)
frequencies = lib_df["BUDGET_CATEGORY"].value_counts(sort=False)
stats_df = pd.DataFrame({"Group": frequencies.index, "Library count": frequencies.values})
stats_df["Library percent"] = stats_df.apply(lambda row: row["Library count"] / lib_count * 100, axis=1)
stats_df.style.format(precision=2, thousands=",")
# category_frequencies.plot.pie(y=category_frequencies.index, autopct='%1.1f%%')

,Group,Library count,Library percent
0,Below $200K,"3,839",41.51
1,$5M - $10M,305,3.30
2,$400K - $1M,"1,642",17.76
3,$200K - $400K,"1,373",14.85
4,$1M - $5M,"1,772",19.16
5,$10M - $30M,162,1.75
6,$30M+,83,0.90
7,None,72,0.78


Next we will calculate and display the population served for each budget category

In [21]:
total_population_served = lib_df[lib_df["POPU_LSA"] > 0]["POPU_LSA"].sum()
print(f"Total population served by libraries in this dataset: {total_population_served:,}")

def get_population_served(df, group):
  return df[(df["POPU_LSA"] > 0) & (df["BUDGET_CATEGORY"] == group)]["POPU_LSA"].sum()

stats_df["Population served"] = stats_df.apply(lambda row: get_population_served(lib_df, row["Group"]), axis=1)
stats_df["% population served"] = stats_df.apply(lambda row: row["Population served"] / total_population_served * 100, axis=1)
stats_df.style.format(precision=2, thousands=",")


Total population served by libraries in this dataset: 334,210,711


,Group,Library count,Library percent,Population served,% population served
0,Below $200K,"3,839",41.51,"11,664,005",3.49
1,$5M - $10M,305,3.30,"42,840,040",12.82
2,$400K - $1M,"1,642",17.76,"28,479,207",8.52
3,$200K - $400K,"1,373",14.85,"13,107,111",3.92
4,$1M - $5M,"1,772",19.16,"90,785,270",27.16
5,$10M - $30M,162,1.75,"50,187,204",15.02
6,$30M+,83,0.90,"94,232,698",28.20
7,None,72,0.78,"2,915,176",0.87


As you can see, the **libraries with the largest budgets ($30M+) account for less than 1% of all libraries but serve the largest percent of the population at 28.2%**.  Just behind that group are ibraries with a budget between $1 and $5M, which serve 27.16% of the population.